In [1]:
import numpy as np
from matplotlib import pyplot as plt

plt.rcParams["figure.figsize"] = (10, 8)
plt.rcParams.update({'font.size': 14})

from cosmoprimo import *

from densitysplit import catalog_data, density_split
from densitysplit.split_model import *
from densitysplit.corr_func_utils import *

# To activate logging
setup_logging()

## Cosmology

In [2]:
z = 1.175

# Abacus summit base cosmology
cosmo_abacus = fiducial.AbacusSummitBase()
bg = cosmo_abacus.get_background()

k=np.logspace(-5, 2, 100000)

## Gaussian mock to compare to model

In [3]:
output_dir = '/feynman/work/dphp/mp270220/outputs/'

mock_catalog = catalog_data.Data.load(output_dir+'AbacusSummit_1Gpc_z1.175_gaussianMock.npy')
mock_catalog_nbarx5 = catalog_data.Data.load(output_dir+'AbacusSummit_1Gpc_z1.175_gaussianMock_nbarx5.npy')

nbar = mock_catalog.size/mock_catalog.boxsize**3
nbarx5 = mock_catalog_nbarx5.size/mock_catalog_nbarx5.boxsize**3

In [4]:
cellsize = 10
resampler = 'tsc'

In [5]:
mocks_list = list()
mocks_density_list = list()
density_bins_list = list()

for i in range(10):
    mock_catalog = catalog_data.Data.load(output_dir+'mocks_rsd/AbacusSummit_1Gpc_z1.175_gaussianMock{}_nbar0.01467461.npy'.format(i))
    mocks_list.append(mock_catalog)
    mock_density = density_split.DensitySplit(mock_catalog)
    mock_density.compute_density(cellsize=cellsize, resampler=resampler, use_rsd=False, use_weights=True)
    mock_density.split_density(3)
    mocks_density_list.append(mock_density)
    density_bins_list.append(mock_density.split_bins)

[000002.40]  06-09 15:30  CatalogMesh                  INFO     Slab 0 ~ 4194304 / 14674610.
[000003.56]  06-09 15:30  CatalogMesh                  INFO     Painted 4194304 out of 14674610 objects to mesh.
[000003.56]  06-09 15:30  CatalogMesh                  INFO     Slab 4194304 ~ 8388608 / 14674610.
[000004.71]  06-09 15:30  CatalogMesh                  INFO     Painted 8388608 out of 14674610 objects to mesh.
[000004.71]  06-09 15:30  CatalogMesh                  INFO     Slab 8388608 ~ 12582912 / 14674610.
[000005.87]  06-09 15:30  CatalogMesh                  INFO     Painted 12582912 out of 14674610 objects to mesh.
[000005.87]  06-09 15:30  CatalogMesh                  INFO     Slab 12582912 ~ 16777216 / 14674610.
[000006.46]  06-09 15:30  CatalogMesh                  INFO     Painted 14674610 out of 14674610 objects to mesh.
[000012.09]  06-09 15:30  CatalogMesh                  INFO     Slab 0 ~ 4194304 / 14674610.
[000013.30]  06-09 15:30  CatalogMesh                  INFO 

In [6]:
mean_density_bins = np.mean(np.array(density_bins_list), axis=0)

In [7]:
mean_density_bins

array([-2.80055491, -0.22859596,  0.2432934 ,  3.00685794])

### Compute density

In [ ]:
# Compute density without weights
mock_density = density_split.DensitySplit(mock_catalog)
mock_density.compute_density(cellsize=cellsize, resampler=resampler, use_rsd=False, use_weights=True)

mock_density_nbarx5 = density_split.DensitySplit(mock_catalog_nbarx5)
mock_density_nbarx5.compute_density(cellsize=cellsize, resampler=resampler, use_rsd=False, use_weights=True)

[000124.78]  06-09 15:32  CatalogMesh                  INFO     Slab 0 ~ 4194304 / 14674610.
[000125.97]  06-09 15:32  CatalogMesh                  INFO     Painted 4194304 out of 14674610 objects to mesh.
[000125.97]  06-09 15:32  CatalogMesh                  INFO     Slab 4194304 ~ 8388608 / 14674610.
[000127.15]  06-09 15:32  CatalogMesh                  INFO     Painted 8388608 out of 14674610 objects to mesh.
[000127.15]  06-09 15:32  CatalogMesh                  INFO     Slab 8388608 ~ 12582912 / 14674610.
[000128.33]  06-09 15:32  CatalogMesh                  INFO     Painted 12582912 out of 14674610 objects to mesh.
[000128.33]  06-09 15:32  CatalogMesh                  INFO     Slab 12582912 ~ 16777216 / 14674610.
[000128.94]  06-09 15:32  CatalogMesh                  INFO     Painted 14674610 out of 14674610 objects to mesh.
[000134.04]  06-09 15:32  CatalogMesh                  INFO     Slab 0 ~ 4194304 / 14674610.
[000136.18]  06-09 15:32  CatalogMesh                  INFO 

### Plot density distribution

In [ ]:
plt.hist(mock_density.data_densities, bins=60, density=True, alpha=0.5, label = r'10 Gaussian mocks ($\bar{{n}}$ = {})'.format(0.003))
plt.hist(mock_density_nbarx5.data_densities, bins=60, density=True, alpha=0.5, label = r'10 Gaussian mocks ($\bar{{n}}$ = {})'.format(0.015))
plt.xlabel("$\delta$")
plt.ylabel("Normalized counts")
plt.show()

## Split model

In [ ]:
model1 = SplitCCFModel(k, redshift=z, cosmology=cosmo_abacus, bias=1.8, nsplits=1)
model = SplitCCFModel(k, redshift=z, cosmology=cosmo_abacus, bias=1.8, nsplits=1, nbar=nbarx5)

In [ ]:
sep = np.linspace(1., 150., 150)

plt.plot(sep, sep**2 * model.xi(sep), label='R = 0 Mpc/h')

for smoothing_scale in [5, 10, 15]:
    model.set_smoothing_scale(smoothing_scale)
    model.compute_xi_R(sep=sep)
    plt.plot(sep, sep**2 * model.xi_R(sep), label='R = {} Mpc/h'.format(smoothing_scale))

plt.xlabel(r'$s$ [$\mathrm{Mpc}/h$]')
plt.ylabel(r'$s^2 \xi_R(s)$ [$(\mathrm{Mpc}/h)^{2}$]')
plt.legend()
plt.show()

In [ ]:
model.set_smoothing_scale(smoothing_scale=10)

In [ ]:
mock_sigma_RR = np.std(mock_density_nbarx5.data_densities)
mock_sigma_RR

In [ ]:
model.sigma_RR

In [ ]:
model.sigma_R

In [ ]:
mock_density_nbarx5.split_density(3)

In [ ]:
mock_density_nbarx5.split_bins

In [ ]:
mean_density_bins

## Compute density split CCFs (randoms-tracers)

In [ ]:
ccf_randoms_tracers = model.ccf_randoms_tracers(density_bins=mean_density_bins)
ccf_randoms_tracers1 = model1.ccf_randoms_tracers(density_bins=mean_density_bins)

In [ ]:
plt.rcParams["figure.figsize"] = (20, 5)
fig, axes = plt.subplots(1, 3, sharex=False, sharey=False)

s = model.sep

for i in range(model.nsplits):
    axes[i].plot(s, s**2 * ccf_randoms_tracers[i], label='With shot noise')
    axes[i].plot(s, s**2 * ccf_randoms_tracers1[i], label='No shot noise')
    axes[i].grid(True)
    axes[i].set_title('DS{}'.format(i+1))    
    axes[i].set_xlabel(r'$s$ [$\mathrm{Mpc}/h$]')

axes[0].set_ylabel(r'$s^2 \xi_{\ell}(s)$ [$(\mathrm{Mpc}/h)^{2}$]')
axes[2].legend()

plt.suptitle('Cross-correlation randoms-halos', y=1.05)
plt.show()

In [ ]:
mocks_split_results = np.load(output_dir+'AbacusSummit_1Gpc_z1.175'+'_10_gaussianMocks_densitySplit_rh_CCF_cellsize10_randomsize4.npy', allow_pickle=True)
mocks_split_results_nbarx5 = np.load(output_dir+'AbacusSummit_1Gpc_z1.175'+'_10_gaussianMocks_nbarx5_densitySplit_rh_CCF_cellsize10_randomsize4.npy', allow_pickle=True)
mocks_xiell_rh, cov_rh = get_split_poles(mocks_split_results, ells=[0], nsplits=model.nsplits)
mocks_xiell_rh_nbarx5, cov_rh_nbarx5 = get_split_poles(mocks_split_results_nbarx5, ells=[0], nsplits=model.nsplits)
std_rh = np.array_split(np.array(np.array_split(np.diag(cov_rh)**0.5, 1)), model.nsplits, axis=1)
std_rh_nbarx5 = np.array_split(np.array(np.array_split(np.diag(cov_rh_nbarx5)**0.5, 1)), model.nsplits, axis=1)

In [ ]:
plt.rcParams["figure.figsize"] = (20, 5)
fig, axes = plt.subplots(1, 3, sharex=False, sharey=False)

s, _, res = mocks_split_results[0][0].get_corr(return_sep=True)
sep = model.sep

for i in range(model.nsplits):
    axes[i].plot(sep, sep**2 * ccf_randoms_tracers[i], label = 'Split model (with shot noise)')
    axes[i].plot(sep, sep**2 * ccf_randoms_tracers1[i], label = 'Split model (no shot noise)')
    #axes[i].errorbar(s, s**2 * mocks_xiell_rh[i][0], s**2 * std_rh[0], label = r'10 Gaussian mocks ($\bar{{n}}$ = {})'.format(0.003))
    axes[i].errorbar(s, s**2 * mocks_xiell_rh_nbarx5[i][0], s**2 * std_rh_nbarx5[0], label = r'10 Gaussian mocks ($\bar{{n}}$ = {})'.format(0.015))
    axes[i].grid(True)
    axes[i].set_title('DS{}'.format(i+1))    
    axes[i].set_xlabel(r'$s$ [$\mathrm{Mpc}/h$]')

axes[0].set_ylabel(r'$s^2 \xi(s)$ [$(\mathrm{Mpc}/h)^{2}$]')
axes[2].legend()

plt.suptitle('Cross-correlation randoms-halos', y=1.05)
plt.show()

## Compute density split CCFs (tracers-tracers)

In [ ]:
ccf_tracers = model.ccf_tracers(density_bins=mean_density_bins)
ccf_tracers1 = model1.ccf_tracers(density_bins=mean_density_bins)

In [ ]:
plt.rcParams["figure.figsize"] = (20, 5)
fig, axes = plt.subplots(1, 3, sharex=False, sharey=False)

sep = model.sep

for i in range(model.nsplits):
    axes[i].plot(sep, sep**2 * ccf_tracers[i], label='With shot noise')
    axes[i].plot(sep, sep**2 * ccf_tracers1[i], label='No shot noise')
    axes[i].grid(True)
    axes[i].set_title('DS{}'.format(i+1))    
    axes[i].set_xlabel(r'$s$ [$\mathrm{Mpc}/h$]')

axes[0].set_ylabel(r'$s^2 \xi_{\ell}(s)$ [$(\mathrm{Mpc}/h)^{2}$]')
axes[2].legend()

plt.suptitle('Cross-correlation halos-halos', y=1.05)
plt.show()

In [ ]:
mocks_split_results = np.load(output_dir+'AbacusSummit_1Gpc_z1.175'+'_10_gaussianMocks_densitySplit_hh_CCF_cellsize10_randomsize4.npy', allow_pickle=True)
mocks_split_results_nbarx5 = np.load(output_dir+'AbacusSummit_1Gpc_z1.175'+'_10_gaussianMocks_nbarx5_densitySplit_hh_CCF_cellsize10_randomsize4.npy', allow_pickle=True)
mocks_xiell_hh, cov_hh = get_split_poles(mocks_split_results, ells=[0], nsplits=model.nsplits)
mocks_xiell_hh_nbarx5, cov_hh_nbarx5 = get_split_poles(mocks_split_results_nbarx5, ells=[0], nsplits=model.nsplits)
std_hh = np.array_split(np.array(np.array_split(np.diag(cov_hh)**0.5, 1)), model.nsplits, axis=1)
std_hh_nbarx5 = np.array_split(np.array(np.array_split(np.diag(cov_hh_nbarx5)**0.5, 1)), model.nsplits, axis=1)

In [ ]:
plt.rcParams["figure.figsize"] = (20, 5)
fig, axes = plt.subplots(1, 3, sharex=False, sharey=False)

s, _, res = mocks_split_results[0][0].get_corr(return_sep=True)
sep = model.sep

for i in range(model.nsplits):
    axes[i].plot(sep, sep**2 * ccf_tracers[i], label = 'Split model (with shot noise)')
    axes[i].plot(sep, sep**2 * ccf_tracers1[i], label = 'Split model (no shot noise)')
    #axes[i].errorbar(s, s**2 * mocks_xiell_hh[i][0], s**2 * std_hh[0], label = r'10 Gaussian mocks ($\bar{{n}}$ = {})'.format(0.003))
    axes[i].errorbar(s, s**2 * mocks_xiell_hh_nbarx5[i][0], s**2 * std_hh_nbarx5[0], label = r'10 Gaussian mocks ($\bar{{n}}$ = {})'.format(0.015))
    axes[i].grid(True)
    axes[i].set_title('DS{}'.format(i+1))    
    axes[i].set_xlabel(r'$s$ [$\mathrm{Mpc}/h$]')

axes[0].set_ylabel(r'$s^2 \xi(s)$ [$(\mathrm{Mpc}/h)^{2}$]')
axes[2].legend()

plt.suptitle('Cross-correlation halos-halos', y=1.05)
plt.show()